In [19]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# Comando simple para leer cualquier hoja usando .env
import os
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configuración desde .env
SERVICE_ACCOUNT_FILE = os.getenv('GOOGLE_SERVICE_ACCOUNT_FILE')
SHEET_ID = os.getenv('GOOGLE_SHEET_ID')
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Autenticación
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Función mejorada para leer TODA la hoja dinámicamente
def leer_hoja(nombre_hoja='REGISTRO_DIARIO'):
    """
    Lee TODA la hoja completa, sin importar cuántas filas tenga
    Se adapta automáticamente al crecimiento de datos
    Maneja celdas vacías al final de las filas
    """
    try:
        # Opción 1: Leer toda la hoja (sin especificar rango)
        range_name = f'{nombre_hoja}'  # Solo el nombre de la hoja
        
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID, 
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            if len(values) > 1:
                # Obtener encabezados y número de columnas esperadas
                headers = values[0]
                num_columns = len(headers)
                
                # Normalizar todas las filas para que tengan el mismo número de columnas
                data_rows = []
                for row in values[1:]:
                    # Si la fila tiene menos columnas, completar con cadenas vacías
                    if len(row) < num_columns:
                        row_completa = row + [''] * (num_columns - len(row))
                        data_rows.append(row_completa)
                    else:
                        data_rows.append(row)
                
                df = pd.DataFrame(data_rows, columns=headers)
            else:
                df = pd.DataFrame(values)
            
            print(f"✅ Se leyeron {len(df)} filas de {nombre_hoja}")
            print(f"📊 Columnas: {list(df.columns)}")
            print(f"🔄 Rango automático: A1:{chr(65+len(df.columns)-1)}{len(df)+1}")
            return df
        else:
            print("❌ No se encontraron datos")
            return None
            
    except Exception as e:
        print(f"❌ Error al leer la hoja: {e}")
        return None

# Leer la hoja REGISTRO_DIARIO (dinámicamente)
print("🔄 Leyendo hoja completa dinámicamente...")
df1 = leer_hoja('REGISTRO_DIARIO')
df2 = leer_hoja('HORARIOLABORAL')


🔄 Leyendo hoja completa dinámicamente...
✅ Se leyeron 1462 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']
🔄 Rango automático: A1:M1463
✅ Se leyeron 1462 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']
🔄 Rango automático: A1:M1463
✅ Se leyeron 20 filas de HORARIOLABORAL
📊 Columnas: ['DNI', 'Colaborador', 'dias', 'hora_entrada', 'hora_salida', 'ID']
🔄 Rango automático: A1:F21
✅ Se leyeron 20 filas de HORARIOLABORAL
📊 Columnas: ['DNI', 'Colaborador', 'dias', 'hora_entrada', 'hora_salida', 'ID']
🔄 Rango automático: A1:F21


In [21]:
df3=leer_hoja("REGISTRO_CALENDARIO")
df4=leer_hoja("PAGOS")
df5=leer_hoja("PAGOSCHECK")

✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
✅ Se leyeron 36 filas de PAGOS
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
🔄 Rango automático: A1:L37
✅ Se leyeron 36 filas de PAGOS
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
🔄 Rango automático: A1:L37
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador'

In [22]:
print(df5)

        Colaborador periodo_inicio periodo_fin  fecha_pago  check
0      Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
1      Andrea Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
2      Andrea Yuqui     16/08/2025  15/09/2025  16/09/2025       
3          Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo
4          Benjamin     02/06/2025  01/07/2025  02/07/2025  Listo
5          Benjamin     02/07/2025  01/08/2025  02/08/2025  Listo
6          Benjamin     02/08/2025  01/09/2025  04/09/2025       
7      Edison Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
8      Edison Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
9      Edison Yuqui     16/08/2025  15/09/2025  16/09/2025       
10         Fiorella     02/06/2025  01/07/2025  02/07/2025  Listo
11         Fiorella     02/07/2025  01/08/2025  02/08/2025  Listo
12             Gary     02/06/2025  01/07/2025  02/07/2025  Listo
13             Gary     02/07/2025  01/08/2025  02/08/2025  Listo
14        

In [23]:
import pandasql as psql
from datetime import datetime
query2="""Select Colaborador,periodo_inicio,periodo_fin, fecha_pago
from df4
Group By Colaborador,fecha_pago
"""
periodos=psql.sqldf(query2,locals())
periodos["check"]=""
periodos.loc[0,"fecha_pago"]
filascount=len(periodos) 
hoy=datetime.today()
for i in range(filascount):
    fecha=periodos.loc[i,"fecha_pago"]
    fecha=datetime.strptime(fecha, "%d/%m/%Y")
    if fecha<hoy:
        periodos.loc[i,"check"]="Listo"

print(periodos)
df5=periodos


        Colaborador periodo_inicio periodo_fin  fecha_pago  check
0      Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
1      Andrea Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
2      Andrea Yuqui     16/08/2025  15/09/2025  16/09/2025       
3          Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo
4          Benjamin     02/06/2025  01/07/2025  02/07/2025  Listo
5          Benjamin     02/07/2025  01/08/2025  02/08/2025  Listo
6          Benjamin     02/08/2025  01/09/2025  02/09/2025  Listo
7      Edison Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
8      Edison Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
9      Edison Yuqui     16/08/2025  15/09/2025  16/09/2025       
10         Fiorella     02/06/2025  01/07/2025  02/07/2025  Listo
11         Fiorella     02/07/2025  01/08/2025  02/08/2025  Listo
12             Gary     02/06/2025  01/07/2025  02/07/2025  Listo
13             Gary     02/07/2025  01/08/2025  02/08/2025  Listo
14        

In [24]:
import pandasql as psql
query="""Select Colaborador
from df3
group by Colaborador
"""

result=psql.sqldf(query,locals())
colaboradores=result["Colaborador"].tolist()
print(colaboradores[0])

for colaborador in colaboradores:
    query=f"""Select *
    from df3
    where Colaborador='{colaborador}'
    """
    result=psql.sqldf(query,locals())
    #result.to_excel(f"REGISTRO_CALENDARIO_{colaborador}.xlsx")
    print(result)

Andrea Yuqui
     Colaborador HoraEntrada    HoraSalida FechaEntrada FechaSalida Minutos  \
0   Andrea Yuqui    17:38:34       0:03:04    11/7/2025   12/7/2025     385   
1   Andrea Yuqui    16:00:33      20:02:18    12/7/2025   12/7/2025     242   
2   Andrea Yuqui     8:01:19  00:03:19.000    13/7/2025   14/7/2025     962   
3   Andrea Yuqui    15:59:51       0:07:52    14/7/2025   15/7/2025     488   
4   Andrea Yuqui    16:00:51       0:08:00    15/7/2025   16/7/2025     487   
5   Andrea Yuqui    16:01:03       0:02:13    16/7/2025   17/7/2025     481   
6   Andrea Yuqui    16:00:10       0:13:19    17/7/2025   18/7/2025     493   
7   Andrea Yuqui    16:03:07       0:02:40    18/7/2025   19/7/2025     480   
8   Andrea Yuqui    16:21:06       0:01:18    19/7/2025   20/7/2025     460   
9   Andrea Yuqui     7:01:36       0:02:40    20/7/2025   21/7/2025    1021   
10  Andrea Yuqui    16:00:56       0:32:59    21/7/2025   22/7/2025     512   
11  Andrea Yuqui    16:00:37       0:03

In [25]:
type(result.loc[0,'FechaEntrada'])
result=result.values.tolist()

In [26]:

for colaborador in colaboradores: 
    for periodo in periodos:
        if colaborador == "Andrea Yuqui" and periodo[0]=="Andrea Yuqui":
            
            query=f"""Select *
                from df3
                where Colaborador='{colaborador}'
            """
            result=psql.sqldf(query,locals())

            result
            print(f"Colaborador {colaborador} encontrado en el periodo {periodo[1]} - {periodo[2]}.")
print(result)


[['Valentino', '19:42:56', '19:43:18', '2/6/2025', '2/6/2025', '0', '0', '0', '1', '', '', '', 'No'], ['Valentino', '8:59:32', '9:24:59', '5/6/2025', '5/6/2025', '25', '0', '25', '9', '', 'Comparacion de los stocks de peri collection y investigacion de los pasos para que el stock de mariana actualice correctamente el estado de los pedidos', '', 'No'], ['Valentino', '18:25:43', '18:31:27', '5/6/2025', '5/6/2025', '6', '0', '6', '10', '', 'Soporte a andre', '', 'No'], ['Valentino', '11:12:13', '11:13:11', '6/6/2025', '6/6/2025', '1', '0', '1', '11', '', 'Me olvide marcar desde las 9:50. Hice soporte a Andrea y avancé con lo de añadir las nuevas tallas a peri collection', '', 'No'], ['Valentino', '13:28:08', '13:29:03', '7/6/2025', '7/6/2025', '1', '0', '1', '13', '', '', '', 'No'], ['Valentino', '15:05:58', '15:51:04', '7/6/2025', '7/6/2025', '45', '0', '45', '19', '', 'Avance del filtro para QRs para Mariana', '', 'No'], ['Valentino', '15:59:45', '18:14:01', '7/6/2025', '7/6/2025', '134

In [27]:
# 📦 INSTALAR LIBRERÍAS
%pip install pandasql openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
df4

,id_pago,DNI,Colaborador,periodo_inicio,periodo_fin,fecha_pago,horas_normales,horas_extra,horas_extra_no_pagadas,pago_base,monto_total,horas_deberia
0,1,29,Mariana Benites,08/08/2025,07/09/2025,08/09/2025,"102,3",0,"14,93",1100,1100,332
1,2,30,Renata Rojas,28/07/2025,27/08/2025,28/08/2025,"37,3",0,"10,77",700,700,156
2,3,31,Keiner NIeves,05/08/2025,19/08/2025,20/08/2025,"35,85",0,"4,63",1200,1200,104
3,4,31,Keiner NIeves,20/08/2025,04/09/2025,05/09/2025,"71,58",0,"5,72",1200,1200,112
4,5,32,Stephany Perez,05/08/2025,19/08/2025,20/08/2025,"52,88",0,"0,63",1000,1000,104
5,6,32,Stephany Perez,20/08/2025,04/09/2025,05/09/2025,"87,03",0,"1,27",1000,1000,112
6,7,33,Andrea Yuqui,16/06/2025,15/07/2025,16/07/2025,"42,32",0,"0,42",0,"46,55","279,5"
7,8,33,Andrea Yuqui,16/07/2025,15/08/2025,16/08/2025,"276,68",0,"12,6",0,"304,35","279,48"
8,9,33,Andrea Yuqui,16/08/2025,15/09/2025,16/09/2025,"165,08",0,"2,43",0,"181,59","287,48"
9,10,34,Edison Yuqui,16/06/2025,15/07/2025,16/07/2025,"35,72",0,"3,45",0,"37,86","195,5"


In [29]:
periodos=leer_hoja("PAGOSCHECK")

✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37


In [30]:
df3.head()

,Colaborador,HoraEntrada,HoraSalida,FechaEntrada,FechaSalida,Minutos,Minutos_extras,Minutos_normales,ID_Calendario,Alerta,Descripcion,Observacion,Extratime
0,Valentino,19:42:56,19:43:18,2/6/2025,2/6/2025,0,0,0,1,,,,No
1,Benjamin,18:58:53,19:45:02,2/6/2025,2/6/2025,46,0,46,2,,,,No
2,Benjamin,14:49:23,15:04:42,3/6/2025,3/6/2025,15,0,15,3,,,,No
3,Fiorella,14:54:17,15:04:49,3/6/2025,3/6/2025,11,0,11,4,,,,No
4,Benjamin,15:38:15,15:40:44,3/6/2025,3/6/2025,2,0,2,5,,,,No


In [31]:
filas=[0,3]
periodos.iloc[filas]
print(periodos.iloc[filas])

    Colaborador periodo_inicio periodo_fin  fecha_pago  check
0  Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
3      Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo


In [32]:
# 🎯 SISTEMA COMPLETO DE PROCESAMIENTO DE ASISTENCIA CON ENVÍO DE CORREOS
import pandas as pd
import pandasql as psql
from datetime import datetime
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv

def procesar_asistencia_completo():
    """
    Sistema completo que:
    1. Verifica si hoy es fecha de pago
    2. Genera reportes de Excel para colaboradores
    3. Envía correos automáticamente
    """
    
    print("🚀 === INICIANDO SISTEMA DE ASISTENCIA ===")
    
    # 📅 VERIFICAR FECHAS DE PAGO
    print("\n📅 Verificando fechas de pago...")
    hoy = datetime.today()
    fecha_hoy_str = hoy.strftime("%d/%m/%Y")
    
    # Leer datos de pago
    df_pagos_check = leer_hoja('PAGOSCHECK')
    if df_pagos_check is None:
        print("❌ Error: No se pudo leer la hoja PAGOSCHECK")
        return False
    
    # Verificar si hoy es fecha de pago
    fechas_pago_hoy = df_pagos_check[df_pagos_check['fecha_pago'] == fecha_hoy_str]
    
    if fechas_pago_hoy.empty:
        print(f"ℹ️  Hoy ({fecha_hoy_str}) no es fecha de pago. No se procesarán reportes.")
        print("   Fechas de pago programadas:")
        for idx, row in df_pagos_check.iterrows():
            print(f"   • {row['Colaborador']}: {row['fecha_pago']}")
        return False
    
    print(f"✅ Hoy ({fecha_hoy_str}) SÍ es fecha de pago para:")
    colaboradores_a_procesar = []
    for idx, row in fechas_pago_hoy.iterrows():
        colaborador = row['Colaborador']
        fecha_pago = row['fecha_pago']
        periodo_inicio = row['periodo_inicio']
        periodo_fin = row['periodo_fin']
        
        print(f"   • {colaborador} - Período: {periodo_inicio} a {periodo_fin}")
        colaboradores_a_procesar.append({
            'colaborador': colaborador,
            'fecha_pago': fecha_pago,
            'periodo_inicio': periodo_inicio,
            'periodo_fin': periodo_fin
        })
    
    # 📊 CARGAR DATOS NECESARIOS
    print("\n📊 Cargando datos de Google Sheets...")
    df_calendario = leer_hoja('REGISTRO_CALENDARIO')
    df_vendedoras = leer_hoja('VENDEDORAS')
    
    if df_calendario is None or df_vendedoras is None:
        print("❌ Error cargando datos necesarios")
        return False
    
    # Crear directorio para reportes si no existe
    directorio_reportes = "Reportes_Asistencia"
    if not os.path.exists(directorio_reportes):
        os.makedirs(directorio_reportes)
        print(f"📁 Directorio '{directorio_reportes}' creado")
    
    # 📧 CONFIGURAR CORREO
    load_dotenv()
    gmail_user = os.getenv('GMAIL_USER')
    gmail_password = os.getenv('GMAIL_APP_PASSWORD')
    
    if not gmail_user or not gmail_password:
        print("⚠️  Credenciales de Gmail no configuradas. Solo se generarán archivos.")
        enviar_correos = False
    else:
        enviar_correos = True
        print(f"📧 Correos se enviarán desde: {gmail_user}")
    
    # 🔄 PROCESAR CADA COLABORADOR
    resultados = []
    
    for colaborador_data in colaboradores_a_procesar:
        colaborador = colaborador_data['colaborador']
        fecha_pago = colaborador_data['fecha_pago']
        periodo_inicio = colaborador_data['periodo_inicio']
        periodo_fin = colaborador_data['periodo_fin']
        
        print(f"\n🔄 Procesando: {colaborador}")
        
        # Filtrar datos del colaborador en el período
        query_asistencia = f"""
        SELECT * 
        FROM df_calendario 
        WHERE Colaborador = '{colaborador}'
        AND FechaEntrada BETWEEN '{periodo_inicio}' AND '{periodo_fin}'
        ORDER BY FechaEntrada
        """
        
        df_asistencia = psql.sqldf(query_asistencia, locals())
        
        if df_asistencia.empty:
            print(f"   ⚠️  No hay registros de asistencia para {colaborador}")
            continue
        
        # Generar reporte Excel
        nombre_archivo = f"Reporte_Asistencia_{colaborador.replace(' ', '_')}_{fecha_pago.replace('/', '-')}.xlsx"
        ruta_archivo = os.path.join(directorio_reportes, nombre_archivo)
        
        if generar_reporte_excel(df_asistencia, colaborador, fecha_pago, periodo_inicio, periodo_fin, ruta_archivo):
            print(f"   ✅ Excel generado: {nombre_archivo}")
            
            # Enviar correo si está configurado
            if enviar_correos:
                email_colaborador = obtener_email_colaborador(colaborador, df_vendedoras)
                
                if email_colaborador:
                    print(f"   📧 Enviando correo a: {email_colaborador}")
                    
                    if enviar_correo_reporte(email_colaborador, colaborador, ruta_archivo, fecha_pago):
                        print(f"   ✅ Correo enviado exitosamente")
                        estado_correo = "Enviado"
                    else:
                        print(f"   ❌ Error enviando correo")
                        estado_correo = "Error"
                else:
                    print(f"   ⚠️  Email no encontrado para {colaborador}")
                    estado_correo = "Sin email"
            else:
                estado_correo = "No configurado"
            
            resultados.append({
                'Colaborador': colaborador,
                'Fecha_Pago': fecha_pago,
                'Archivo_Excel': nombre_archivo,
                'Estado_Correo': estado_correo,
                'Email': email_colaborador if enviar_correos else 'N/A'
            })
        else:
            print(f"   ❌ Error generando Excel para {colaborador}")
    
    # 📋 RESUMEN FINAL
    print(f"\n📋 === RESUMEN DE PROCESAMIENTO ===")
    print(f"✅ Colaboradores procesados: {len(resultados)}")
    
    if resultados:
        df_resultados = pd.DataFrame(resultados)
        print("\n📊 Detalle:")
        print(df_resultados.to_string(index=False))
        
        # Guardar resumen
        resumen_archivo = os.path.join(directorio_reportes, f"Resumen_Procesamiento_{fecha_hoy_str.replace('/', '-')}.xlsx")
        df_resultados.to_excel(resumen_archivo, index=False)
        print(f"\n💾 Resumen guardado en: {resumen_archivo}")
    
    print("\n🎉 ¡Procesamiento completado!")
    return True

def generar_reporte_excel(df_asistencia, colaborador, fecha_pago, periodo_inicio, periodo_fin, archivo_salida):
    """
    Genera un archivo Excel con el reporte de asistencia
    """
    try:
        with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
            # Hoja 1: Detalle de Asistencia
            df_asistencia.to_excel(writer, sheet_name='Asistencia', index=False)
            
            # Hoja 2: Resumen
            resumen_data = {
                'Colaborador': [colaborador],
                'Período Inicio': [periodo_inicio],
                'Período Fin': [periodo_fin],
                'Fecha de Pago': [fecha_pago],
                'Total Registros': [len(df_asistencia)],
                'Generado el': [datetime.now().strftime("%d/%m/%Y %H:%M")]
            }
            
            pd.DataFrame(resumen_data).to_excel(writer, sheet_name='Resumen', index=False)
        
        return True
        
    except Exception as e:
        print(f"❌ Error generando Excel: {e}")
        return False

def obtener_email_colaborador(colaborador, df_vendedoras):
    """
    Obtiene el email del colaborador desde la hoja VENDEDORAS
    """
    # Buscar columnas de email posibles
    columnas_email = ['Email', 'email', 'Correo', 'correo', 'Gmail', 'gmail']
    columna_encontrada = None
    
    for col in columnas_email:
        if col in df_vendedoras.columns:
            columna_encontrada = col
            break
    
    if not columna_encontrada:
        return None
    
    # Buscar el colaborador
    resultado = df_vendedoras[df_vendedoras['Colaborador'] == colaborador]
    
    if not resultado.empty:
        email = resultado.iloc[0][columna_encontrada]
        if email and str(email).strip() and str(email) != 'nan':
            return str(email).strip()
    
    return None

def enviar_correo_reporte(destinatario, colaborador, archivo_excel, fecha_pago):
    """
    Envía el correo con el reporte adjunto usando EmailMessage (más moderno)
    """
    try:
        # Configuración
        load_dotenv()
        gmail_user = os.getenv('GMAIL_USER')
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')
        
        # Crear mensaje
        msg = EmailMessage()
        msg['Subject'] = f'Reporte de Asistencia - {colaborador} ({fecha_pago})'
        msg['From'] = gmail_user
        msg['To'] = destinatario
        
        # Cuerpo del mensaje
        cuerpo = f"""Estimado/a {colaborador},

Te enviamos tu reporte de asistencia correspondiente al periodo de pago: {fecha_pago}.

En el archivo adjunto encontraras:
• Detalle de tus registros de asistencia
• Fechas de entrada y salida
• Resumen del periodo evaluado

Si tienes alguna consulta, no dudes en contactarnos.

Saludos cordiales,
Equipo de Recursos Humanos

---
Este correo fue generado automaticamente por el Sistema de Asistencia.
"""
        
        msg.set_content(cuerpo)
        
        # Adjuntar archivo Excel
        with open(archivo_excel, 'rb') as f:
            file_data = f.read()
            file_name = os.path.basename(archivo_excel)
        
        msg.add_attachment(file_data, 
                          maintype='application', 
                          subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                          filename=file_name)
        
        # Enviar
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(gmail_user, gmail_password)
            server.send_message(msg)
        
        return True
        
    except Exception as e:
        print(f"❌ Error enviando correo: {e}")
        return False

# Ejecutar el sistema completo
if __name__ == "__main__":
    procesar_asistencia_completo()

🚀 === INICIANDO SISTEMA DE ASISTENCIA ===

📅 Verificando fechas de pago...
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Hoy (04/09/2025) SÍ es fecha de pago para:
   • Benjamin - Período: 02/08/2025 a 01/09/2025

📊 Cargando datos de Google Sheets...
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Hoy (04/09/2025) SÍ es fecha de pago para:
   • Benjamin - Período: 02/08/2025 a 01/09/2025

📊 Cargando datos de Google Sheets...
✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada

In [38]:
# 🔍 DIAGNÓSTICO: Verificar datos antes de procesar
from datetime import datetime

print("🔍 === DIAGNÓSTICO DEL SISTEMA ===")

# 1. Verificar fecha actual
hoy = datetime.today()
fecha_hoy_str = hoy.strftime("%d/%m/%Y")
print(f"📅 Fecha de hoy: {fecha_hoy_str}")

# 2. Verificar datos de PAGOSCHECK
print("\n📋 Verificando PAGOSCHECK...")
try:
    df_pagos_check = leer_hoja('PAGOSCHECK')
    if df_pagos_check is not None:
        print(f"✅ PAGOSCHECK cargado: {len(df_pagos_check)} registros")
        print("📊 Columnas:", list(df_pagos_check.columns))
        print("\n📅 Fechas de pago programadas:")
        for idx, row in df_pagos_check.iterrows():
            print(f"   • {row['Colaborador']}: {row['fecha_pago']} (Período: {row['periodo_inicio']} - {row['periodo_fin']})")
        
        # Verificar si hoy coincide con alguna fecha
        fechas_pago_hoy = df_pagos_check[df_pagos_check['fecha_pago'] == fecha_hoy_str]
        if not fechas_pago_hoy.empty:
            print(f"\n✅ ¡HOY SÍ ES FECHA DE PAGO para {len(fechas_pago_hoy)} colaborador(es)!")
        else:
            print(f"\nℹ️  Hoy ({fecha_hoy_str}) NO es fecha de pago.")
    else:
        print("❌ Error cargando PAGOSCHECK")
except Exception as e:
    print(f"❌ Error: {e}")

# 3. Verificar datos de VENDEDORAS
print("\n📋 Verificando VENDEDORAS...")
try:
    df_vendedoras = leer_hoja('VENDEDORAS')
    if df_vendedoras is not None:
        print(f"✅ VENDEDORAS cargado: {len(df_vendedoras)} registros")
        print("📊 Columnas:", list(df_vendedoras.columns))
        print("\n👥 Colaboradores con email:")
        for idx, row in df_vendedoras.iterrows():
            if 'Email' in df_vendedoras.columns:
                print(f"   • {row['Colaborador']}: {row.get('Email', 'No definido')}")
            else:
                print(f"   • {row['Colaborador']}: (columna Email no encontrada)")
    else:
        print("❌ Error cargando VENDEDORAS")
except Exception as e:
    print(f"❌ Error: {e}")

# 4. Verificar datos de REGISTRO_CALENDARIO  
print("\n📋 Verificando REGISTRO_CALENDARIO...")
try:
    df_calendario = leer_hoja('REGISTRO_CALENDARIO')
    if df_calendario is not None:
        print(f"✅ REGISTRO_CALENDARIO cargado: {len(df_calendario)} registros")
        print("📊 Columnas:", list(df_calendario.columns))
        print(f"👥 Colaboradores únicos: {df_calendario['Colaborador'].nunique()}")
        print("📅 Rango de fechas:", 
              f"desde {df_calendario['FechaEntrada'].min()}" if 'FechaEntrada' in df_calendario.columns else "FechaEntrada no encontrada",
              f"hasta {df_calendario['FechaEntrada'].max()}" if 'FechaEntrada' in df_calendario.columns else "")
    else:
        print("❌ Error cargando REGISTRO_CALENDARIO")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n🎯 ¿Quieres que ajuste la fecha para forzar el procesamiento de prueba?")
print("   Opción 1: Cambiar fecha de hoy a una fecha de pago existente")
print("   Opción 2: Ejecutar el sistema tal como está")
print("   Opción 3: Crear datos de prueba")

🔍 === DIAGNÓSTICO DEL SISTEMA ===
📅 Fecha de hoy: 04/09/2025

📋 Verificando PAGOSCHECK...
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ PAGOSCHECK cargado: 36 registros
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']

📅 Fechas de pago programadas:
   • Andrea Yuqui: 16/07/2025 (Período: 16/06/2025 - 15/07/2025)
   • Andrea Yuqui: 16/08/2025 (Período: 16/07/2025 - 15/08/2025)
   • Andrea Yuqui: 16/09/2025 (Período: 16/08/2025 - 15/09/2025)
   • Benjamin: 02/06/2025 (Período: 02/05/2025 - 01/06/2025)
   • Benjamin: 02/07/2025 (Período: 02/06/2025 - 01/07/2025)
   • Benjamin: 02/08/2025 (Período: 02/07/2025 - 01/08/2025)
   • Benjamin: 04/09/2025 (Período: 02/08/2025 - 01/09/2025)
   • Edison Yuqui: 16/07/2025 (Período: 16/06/2025 - 15/07/2025)
   • Edison Yuqui: 16/08/2025 (Período: 16/07/2025 - 15/08/2025)
   • Edison Yuqui: 16/09/2025 (Período: 16/0

In [ ]:
# 🎯 PROCESAR REPORTES PARA FECHA DE HOY (independiente del estado "listo")
import pandas as pd
import pandasql as psql
from datetime import datetime
import os
from openpyxl import Workbook
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv

def procesar_reportes_hoy():
    """
    Procesa reportes para colaboradores con fecha de pago HOY,
    sin importar si su estado está marcado como "listo"
    """
    
    print("🚀 === PROCESANDO REPORTES PARA HOY ===")
    
    # 📅 OBTENER FECHA DE HOY
    hoy = datetime.today()
    fecha_hoy_str = hoy.strftime("%d/%m/%Y")
    print(f"📅 Fecha actual: {fecha_hoy_str}")
    
    # 📊 CARGAR DATOS
    print("\n📊 Cargando datos de Google Sheets...")
    df_pagos_check = leer_hoja('PAGOSCHECK')
    df_calendario = leer_hoja('REGISTRO_CALENDARIO')
    df_vendedoras = leer_hoja('VENDEDORAS')
    
    if any(df is None for df in [df_pagos_check, df_calendario, df_vendedoras]):
        print("❌ Error cargando datos necesarios")
        return False
    
    # 🔍 ENCONTRAR COLABORADORES CON FECHA DE PAGO HOY
    colaboradores_hoy = df_pagos_check[df_pagos_check['fecha_pago'] == fecha_hoy_str]
    
    if colaboradores_hoy.empty:
        print(f"ℹ️  No hay colaboradores con fecha de pago para hoy ({fecha_hoy_str})")
        print("\n📅 Fechas de pago programadas:")
        for idx, row in df_pagos_check.iterrows():
            estado = "✅ Listo" if row.get('check', '') == 'Listo' else "⏳ Pendiente"
            print(f"   • {row['Colaborador']}: {row['fecha_pago']} - {estado}")
        return False
    
    print(f"\n✅ Encontrados {len(colaboradores_hoy)} colaborador(es) con fecha de pago HOY:")
    for idx, row in colaboradores_hoy.iterrows():
        estado = "✅ Listo" if row.get('check', '') == 'Listo' else "⏳ Pendiente"
        print(f"   • {row['Colaborador']} - Período: {row['periodo_inicio']} a {row['periodo_fin']} - {estado}")
    
    # 📁 CREAR DIRECTORIO
    directorio_reportes = "Reportes_Asistencia"
    if not os.path.exists(directorio_reportes):
        os.makedirs(directorio_reportes)
        print(f"\n📁 Directorio '{directorio_reportes}' creado")
    
    # 📧 CONFIGURAR CORREO
    load_dotenv()
    gmail_user = os.getenv('GMAIL_USER')
    gmail_password = os.getenv('GMAIL_APP_PASSWORD')
    enviar_correos = gmail_user and gmail_password
    
    if enviar_correos:
        print(f"📧 Correos se enviarán desde: {gmail_user}")
    else:
        print("⚠️  Credenciales de Gmail no configuradas. Solo se generarán archivos Excel.")
    
    # 🔄 PROCESAR CADA COLABORADOR
    resultados = []
    
    for idx, row in colaboradores_hoy.iterrows():
        colaborador = row['Colaborador']
        fecha_pago = row['fecha_pago']
        periodo_inicio = row['periodo_inicio']
        periodo_fin = row['periodo_fin']
        
        print(f"\n🔄 Procesando: {colaborador}")
        print(f"   📅 Período: {periodo_inicio} - {periodo_fin}")
        
        # 🔧 MÉTODO CORREGIDO: Convertir fechas antes de comparar
        try:
            # Convertir fechas del período a objetos datetime para comparación
            fecha_inicio_dt = datetime.strptime(periodo_inicio, "%d/%m/%Y")
            fecha_fin_dt = datetime.strptime(periodo_fin, "%d/%m/%Y")
            
            # Filtrar el DataFrame usando pandas en lugar de SQL
            print("   🔍 Filtrando registros de asistencia...")
            
            # Primero filtrar por colaborador
            df_colaborador = df_calendario[df_calendario['Colaborador'] == colaborador].copy()
            
            if df_colaborador.empty:
                print(f"   ⚠️  No hay registros para el colaborador {colaborador}")
                continue
            
            # Convertir FechaEntrada a datetime para comparación
            df_colaborador['FechaEntrada_dt'] = pd.to_datetime(df_colaborador['FechaEntrada'], format="%d/%m/%Y", errors='coerce')
            
            # Filtrar por rango de fechas
            df_asistencia = df_colaborador[
                (df_colaborador['FechaEntrada_dt'] >= fecha_inicio_dt) & 
                (df_colaborador['FechaEntrada_dt'] <= fecha_fin_dt)
            ].copy()
            
            # Eliminar la columna auxiliar
            if 'FechaEntrada_dt' in df_asistencia.columns:
                df_asistencia = df_asistencia.drop('FechaEntrada_dt', axis=1)
            
            if df_asistencia.empty:
                print(f"   ⚠️  No hay registros de asistencia para {colaborador} en el período {periodo_inicio} - {periodo_fin}")
                
                # Mostrar diagnóstico
                print(f"   🔍 Diagnóstico:")
                print(f"     - Total registros del colaborador: {len(df_colaborador)}")
                if len(df_colaborador) > 0:
                    fechas_disponibles = df_colaborador['FechaEntrada'].unique()[:5]  # Mostrar solo las primeras 5
                    print(f"     - Fechas disponibles (muestra): {fechas_disponibles}")
                continue
            
            print(f"   📋 Encontrados {len(df_asistencia)} registros de asistencia")
            
            # 📄 GENERAR ARCHIVO EXCEL (SOLO DEL COLABORADOR)
            nombre_archivo = f"Reporte_Asistencia_{colaborador.replace(' ', '_')}_{fecha_pago.replace('/', '-')}.xlsx"
            ruta_archivo = os.path.join(directorio_reportes, nombre_archivo)
            
            # Crear Excel simple con solo los datos del colaborador
            try:
                df_asistencia.to_excel(ruta_archivo, index=False, engine='openpyxl')
                print(f"   ✅ Excel generado: {nombre_archivo}")
                
                # 📧 ENVIAR CORREO (si está configurado)
                estado_correo = "No configurado"
                email_colaborador = "N/A"
                
                if enviar_correos:
                    # Buscar correo directamente de la columna 'Correo'
                    email_colaborador = obtener_correo_colaborador(colaborador, df_vendedoras)
                    
                    if email_colaborador:
                        print(f"   📧 Enviando correo a: {email_colaborador}")
                        
                        if enviar_correo_con_excel(email_colaborador, colaborador, ruta_archivo, fecha_pago):
                            print(f"   ✅ Correo enviado exitosamente")
                            estado_correo = "Enviado"
                        else:
                            print(f"   ❌ Error enviando correo")
                            estado_correo = "Error"
                    else:
                        print(f"   ⚠️  Email no encontrado para {colaborador}")
                        estado_correo = "Sin email"
                
                # 📝 REGISTRAR RESULTADO
                resultados.append({
                    'Colaborador': colaborador,
                    'Fecha_Pago': fecha_pago,
                    'Archivo_Excel': nombre_archivo,
                    'Estado_Correo': estado_correo,
                    'Email': email_colaborador,
                    'Registros_Asistencia': len(df_asistencia)
                })
                
            except Exception as e:
                print(f"   ❌ Error generando Excel: {e}")
                
        except ValueError as e:
            print(f"   ❌ Error procesando fechas para {colaborador}: {e}")
            print(f"      Período recibido: {periodo_inicio} - {periodo_fin}")
            continue
        except Exception as e:
            print(f"   ❌ Error procesando {colaborador}: {e}")
            continue
    
    # 📋 MOSTRAR RESUMEN
    print(f"\n📋 === RESUMEN DE PROCESAMIENTO ===")
    print(f"✅ Colaboradores procesados: {len(resultados)}")
    
    if resultados:
        for resultado in resultados:
            print(f"   • {resultado['Colaborador']}: {resultado['Archivo_Excel']} - Correo: {resultado['Estado_Correo']}")
    
    print("\n🎉 ¡Procesamiento completado!")
    return len(resultados) > 0

def obtener_correo_colaborador(colaborador, df_vendedoras):
    """
    Obtiene el correo del colaborador desde la columna 'Correo' en VENDEDORAS
    """
    if 'Correo' not in df_vendedoras.columns:
        print(f"❌ No se encontró la columna 'Correo' en VENDEDORAS")
        return None
    
    # Buscar el colaborador
    resultado = df_vendedoras[df_vendedoras['Colaborador'] == colaborador]
    
    if not resultado.empty:
        correo = resultado.iloc[0]['Correo']
        if correo and str(correo).strip() and str(correo) != 'nan':
            return str(correo).strip()
    
    return None

def enviar_correo_con_excel(destinatario, colaborador, archivo_excel, fecha_pago):
    """Envía correo con Excel adjunto"""
    try:
        load_dotenv()
        gmail_user = os.getenv('GMAIL_USER')
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')
        
        # Crear mensaje
        msg = EmailMessage()
        msg['Subject'] = f'Reporte de Asistencia - {colaborador} ({fecha_pago})'
        msg['From'] = gmail_user
        msg['To'] = destinatario
        
        # Cuerpo del mensaje
        cuerpo = f"""Estimado/a {colaborador},

Te enviamos tu reporte de asistencia correspondiente al periodo de pago: {fecha_pago}.

En el archivo adjunto encontraras el detalle de tus registros de asistencia.

Si tienes alguna consulta, no dudes en contactarnos.

Saludos cordiales,
Equipo de Recursos Humanos

---
Este correo fue generado automaticamente por el Sistema de Asistencia.
"""
        
        msg.set_content(cuerpo)
        
        # Adjuntar Excel
        with open(archivo_excel, 'rb') as f:
            file_data = f.read()
            file_name = os.path.basename(archivo_excel)
        
        msg.add_attachment(file_data,
                          maintype='application',
                          subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
                          filename=file_name)
        
        # Enviar
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(gmail_user, gmail_password)
            server.send_message(msg)
        
        return True
        
    except Exception as e:
        print(f"❌ Error enviando correo: {e}")
        return False

# 🚀 EJECUTAR PROCESAMIENTO PARA HOY
print("🎯 Listo para procesar reportes del día de hoy")
print("   Para ejecutar, llama: procesar_reportes_hoy()")

In [40]:
# 🚀 EJECUTAR PROCESAMIENTO DE REPORTES PARA HOY
procesar_reportes_hoy()

🚀 === PROCESANDO REPORTES PARA HOY ===
📅 Fecha actual: 04/09/2025

📊 Cargando datos de Google Sheets...
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
✅ Se leyeron 19 filas de VENDEDORAS
📊 Columnas: ['DNI', 'Colaborador', 'Contraseña', 'Correo', 'Perfil', 'Foto', 'pago_base', 'pago_por_hora', 'Moneda', 'Bono de Orden Taller', 'Tarifa Hora Extra', 'SubPerfil']
🔄 Rango automático: A1:L20

✅ Encontrados 1 colaborador(es) con fecha de pago HOY:
   • Benjamin - Período: 02/08/2025 a 01/09/2025 - ⏳ Pendiente
📧 Correos se enviarán desde: proyectosperi@gmail.com

🔄 Procesando: Benjamin
   ⚠️  No hay registros de asisten

False

NameError: name 'asistencia' is not defined

In [47]:
import pandasql as psql
from datetime import datetime

hoy=datetime.today()
asistencia=leer_hoja("REGISTRO_CALENDARIO")
query="""
SELECT *
FROM asistencia
WHERE Colaborador='Benjamin'
"""
result=psql.sqldf(query,locals())
print(type(result.loc[0,'FechaEntrada']))


✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
<class 'str'>


In [37]:
# 📧 CONFIGURACIÓN DE CORREO ELECTRÓNICO
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv

# Cargar información de vendedoras
df_vendedoras = leer_hoja('VENDEDORAS')
print("📋 Información de vendedoras cargada:")
print(df_vendedoras[['Colaborador', 'Email'] if 'Email' in df_vendedoras.columns else df_vendedoras.columns])

# 📧 FUNCIÓN PARA ENVIAR CORREOS (CORREGIDA)
def enviar_correo_gmail(destinatario_email, colaborador_nombre, archivo_excel, fecha_pago):
    """
    Envía un correo con el reporte de asistencia adjunto
    """
    try:
        # Configuración Gmail desde .env
        load_dotenv()
        gmail_user = os.getenv('GMAIL_USER')  # Tu email de Gmail
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')  # Contraseña de aplicación de Gmail
        
        if not gmail_user or not gmail_password:
            print("❌ Error: Configurar GMAIL_USER y GMAIL_APP_PASSWORD en archivo .env")
            return False
        
        # Crear mensaje
        msg = MIMEMultipart()
        msg['From'] = gmail_user
        msg['To'] = destinatario_email
        msg['Subject'] = f"📊 Reporte de Asistencia - {colaborador_nombre} ({fecha_pago})"
        
        # Cuerpo del correo (texto simple sin caracteres especiales problemáticos)
        cuerpo = f"""Estimado/a {colaborador_nombre},

Te enviamos tu reporte de asistencia correspondiente al periodo de pago: {fecha_pago}.

En el archivo adjunto encontraras:
• Detalle de tus registros de asistencia
• Fechas de entrada y salida
• Informacion del periodo evaluado
• Resumen del reporte

Si tienes alguna consulta sobre tu reporte, no dudes en contactarnos.

Saludos cordiales,
Equipo de Recursos Humanos

---
Este es un correo automatico generado por el Sistema de Asistencia.
"""
        
        # Agregar cuerpo del mensaje con codificación específica
        msg.attach(MIMEText(cuerpo, 'plain', 'utf-8'))
        
        # 🔧 ADJUNTAR ARCHIVO EXCEL (CORREGIDO)
        try:
            # Verificar que el archivo existe
            if not os.path.exists(archivo_excel):
                print(f"❌ Archivo no encontrado: {archivo_excel}")
                return False
            
            with open(archivo_excel, "rb") as attachment:
                part = MIMEBase('application', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet')
                part.set_payload(attachment.read())
            
            # Codificar el archivo en base64
            encoders.encode_base64(part)
            
            # Agregar cabecera para el adjunto
            nombre_archivo_safe = os.path.basename(archivo_excel).encode('ascii', 'ignore').decode('ascii')
            part.add_header(
                'Content-Disposition',
                f'attachment; filename="{nombre_archivo_safe}"'
            )
            msg.attach(part)
            
        except Exception as e:
            print(f"❌ Error al procesar archivo adjunto: {e}")
            return False
        
        # 📧 CONECTAR Y ENVIAR
        try:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()  # Habilitar seguridad
            server.login(gmail_user, gmail_password)
            
            # Convertir mensaje a string
            text = msg.as_string()
            server.sendmail(gmail_user, destinatario_email, text)
            server.quit()
            
            return True
            
        except smtplib.SMTPAuthenticationError:
            print(f"❌ Error de autenticación Gmail. Verificar credenciales en .env")
            return False
        except smtplib.SMTPException as e:
            print(f"❌ Error SMTP: {e}")
            return False
        except Exception as e:
            print(f"❌ Error al conectar con Gmail: {e}")
            return False
        
    except Exception as e:
        print(f"❌ Error general enviando correo a {destinatario_email}: {e}")
        return False

# 🔍 FUNCIÓN PARA OBTENER EMAIL DEL COLABORADOR
def obtener_email_colaborador(nombre_colaborador, df_vendedoras):
    """
    Busca el email del colaborador en la tabla VENDEDORAS
    """
    # Verificar si existe la columna Email (puede tener diferentes nombres)
    posibles_columnas_email = ['Email', 'email', 'Correo', 'correo', 'Gmail', 'gmail']
    columna_email = None
    
    for col in posibles_columnas_email:
        if col in df_vendedoras.columns:
            columna_email = col
            break
    
    if not columna_email:
        print(f"❌ No se encontró columna de email en tabla VENDEDORAS")
        return None
    
    # Buscar el colaborador
    colaborador_info = df_vendedoras[df_vendedoras['Colaborador'] == nombre_colaborador]
    
    if not colaborador_info.empty:
        email = colaborador_info.iloc[0][columna_email]
        if email and str(email).strip() and email != 'nan':
            return str(email).strip()
    
    print(f"⚠️  Email no encontrado para {nombre_colaborador}")
    return None

✅ Se leyeron 19 filas de VENDEDORAS
📊 Columnas: ['DNI', 'Colaborador', 'Contraseña', 'Correo', 'Perfil', 'Foto', 'pago_base', 'pago_por_hora', 'Moneda', 'Bono de Orden Taller', 'Tarifa Hora Extra', 'SubPerfil']
🔄 Rango automático: A1:L20
📋 Información de vendedoras cargada:
         DNI      Colaborador Contraseña                Correo       Perfil  \
0         28       Nitza Peri         28                              Admin   
1         29  Mariana Benites         29                        Colaborador   
2         30     Renata Rojas         30                        Colaborador   
3         31    Keiner NIeves         31                        Colaborador   
4         32   Stephany Perez         32                        Colaborador   
5         33     Andrea Yuqui         33                        Colaborador   
6         34     Edison Yuqui         34                        Colaborador   
7         35       Rosa Clavo         35                        Colaborador   
8         36  

In [34]:
# 🔍 FUNCIÓN DE PRUEBA SÚPER SIMPLIFICADA
def probar_gmail_basico():
    """
    Prueba básica paso a paso para identificar el problema
    """
    try:
        print("1. ⚡ Importando librerías...")
        import smtplib
        from email.mime.text import MIMEText
        from email.mime.multipart import MIMEMultipart
        from dotenv import load_dotenv
        import os
        
        print("2. 📂 Cargando variables de entorno...")
        load_dotenv()
        
        print("3. 🔑 Obteniendo credenciales...")
        gmail_user = os.getenv('GMAIL_USER')
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')
        
        print(f"   Usuario: {gmail_user}")
        print(f"   Password existe: {'Si' if gmail_password else 'No'}")
        
        if not gmail_user or not gmail_password:
            print("❌ Faltan credenciales")
            return False
        
        print("4. 📝 Creando mensaje simple...")
        
        # Mensaje completamente básico
        msg = MIMEMultipart()
        msg['From'] = gmail_user
        msg['To'] = "becueva749@gmail.com"
        msg['Subject'] = "Prueba Simple"
        
        print("5. ✏️ Agregando cuerpo del mensaje...")
        
        # Cuerpo súper simple sin caracteres especiales
        body = "Hola, esto es una prueba simple del sistema."
        
        # Agregar texto sin codificación específica
        msg.attach(MIMEText(body, 'plain'))
        
        print("6. 🌐 Conectando con Gmail...")
        server = smtplib.SMTP('smtp.gmail.com', 587)
        
        print("7. 🔒 Iniciando TLS...")
        server.starttls()
        
        print("8. 👤 Autenticando...")
        server.login(gmail_user, gmail_password)
        
        print("9. 📤 Convirtiendo mensaje...")
        # Aquí es donde probablemente falla
        text = msg.as_string()
        
        print("10. 📨 Enviando...")
        server.sendmail(gmail_user, "becueva749@gmail.com", text)
        
        print("11. ✅ Cerrando conexión...")
        server.quit()
        
        print("🎉 ¡Correo enviado exitosamente!")
        return True
        
    except Exception as e:
        print(f"❌ Error en paso: {e}")
        print(f"   Tipo de error: {type(e)}")
        import traceback
        traceback.print_exc()
        return False

# 🧪 PRUEBA ALTERNATIVA SIN MIMEMultipart
def probar_gmail_simple():
    """
    Prueba con el método más básico posible
    """
    try:
        print("🔧 Prueba con método básico...")
        import smtplib
        from email.mime.text import MIMEText
        from dotenv import load_dotenv
        import os
        
        load_dotenv()
        gmail_user = os.getenv('GMAIL_USER')
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')
        
        # Crear mensaje básico
        mensaje = "Esto es una prueba basica sin caracteres especiales."
        
        # Usar solo MIMEText
        msg = MIMEText(mensaje, 'plain', 'utf-8')
        msg['Subject'] = 'Prueba Basica'
        msg['From'] = gmail_user
        msg['To'] = "becueva749@gmail.com"
        
        # Enviar
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(gmail_user, gmail_password)
        server.send_message(msg)  # Usar send_message en lugar de sendmail
        server.quit()
        
        print("✅ Correo básico enviado exitosamente")
        return True
        
    except Exception as e:
        print(f"❌ Error en método básico: {e}")
        return False

# 🧪 FUNCIÓN DE DIAGNÓSTICO ULTRA SIMPLE
def test_gmail_minimo():
    """
    Prueba mínima con codificación ASCII pura
    """
    try:
        import smtplib
        from email.message import EmailMessage
        from dotenv import load_dotenv
        import os
        
        load_dotenv()
        gmail_user = os.getenv('GMAIL_USER')
        gmail_password = os.getenv('GMAIL_APP_PASSWORD')
        
        # Mensaje con EmailMessage (más moderno)
        msg = EmailMessage()
        msg['Subject'] = 'Test ASCII'
        msg['From'] = gmail_user
        msg['To'] = 'becueva749@gmail.com'
        msg.set_content('Test message with ASCII only characters.')
        
        # Enviar
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(gmail_user, gmail_password)
            server.send_message(msg)
        
        print("✅ Test mínimo exitoso")
        return True
        
    except Exception as e:
        print(f"❌ Error en test mínimo: {e}")
        return False

print("🔧 Funciones de diagnóstico creadas. Para probar, ejecuta:")
print("   probar_gmail_basico()")
print("   probar_gmail_simple()")  
print("   test_gmail_minimo()")

🔧 Funciones de diagnóstico creadas. Para probar, ejecuta:
   probar_gmail_basico()
   probar_gmail_simple()
   test_gmail_minimo()


In [35]:
# 🧪 EJECUTAR PRUEBA ESPECÍFICA
# Descomenta solo UNA línea para probar:

#test_gmail_minimo()          # ← Prueba más simple (recomendada)
probar_gmail_simple()        # ← Prueba básica
#probar_gmail_basico()        # ← Prueba detallada

🔧 Prueba con método básico...
✅ Correo básico enviado exitosamente
✅ Correo básico enviado exitosamente


True

# 📧 CONFIGURACIÓN REQUERIDA PARA GMAIL

Para que funcione el envío de correos, necesitas configurar las siguientes variables en tu archivo `.env`:

```env
# Configuración Gmail
GMAIL_USER=tu-email@gmail.com
GMAIL_APP_PASSWORD=tu-contraseña-de-aplicacion
```

## 🔑 ¿Cómo obtener la contraseña de aplicación de Gmail?

1. **Ir a tu cuenta de Google** → Configuración de seguridad
2. **Activar la verificación en 2 pasos** (si no la tienes)
3. **Generar una contraseña de aplicación**:
   - Ve a "Contraseñas de aplicaciones"
   - Selecciona "Correo" y "Otro"
   - Copia la contraseña de 16 caracteres generada
4. **Agregar al archivo .env** sin espacios ni guiones

⚠️ **IMPORTANTE**: Usa la contraseña de aplicación, NO tu contraseña normal de Gmail.

In [36]:
# 🔐 CONFIGURAR CREDENCIALES DE GMAIL EN ARCHIVO .env
import os

# Verificar si existe el archivo .env
env_path = '.env'
env_exists = os.path.exists(env_path)

print(f"📄 Archivo .env {'existe' if env_exists else 'no existe'}")

# Leer contenido actual del .env (si existe)
existing_content = ""
if env_exists:
    with open(env_path, 'r') as f:
        existing_content = f.read()
    print("📋 Contenido actual del .env:")
    print(existing_content)

# Configurar las nuevas credenciales de Gmail
gmail_config = """
# Configuración Gmail para envío de reportes
GMAIL_USER=proyectosperi@gmail.com
GMAIL_APP_PASSWORD=nhdz wqvv lxwc mgto
"""

# Si el archivo ya existe, agregar las credenciales
if env_exists:
    # Verificar si ya existen configuraciones de Gmail
    if 'GMAIL_USER' in existing_content or 'GMAIL_APP_PASSWORD' in existing_content:
        print("⚠️  Las credenciales de Gmail ya existen en .env")
        print("   Revisa manualmente el archivo para actualizarlas si es necesario")
    else:
        # Agregar las credenciales al final del archivo
        with open(env_path, 'a') as f:
            f.write(gmail_config)
        print("✅ Credenciales de Gmail agregadas al archivo .env existente")
else:
    # Crear el archivo .env con las credenciales
    with open(env_path, 'w') as f:
        f.write(gmail_config)
    print("✅ Archivo .env creado con credenciales de Gmail")

print("\n🔐 Configuración de Gmail:")
print("   GMAIL_USER: proyectosperi@gmail.com")
print("   GMAIL_APP_PASSWORD: nhdz wqvv lxwc mgto")
print("\n🚀 ¡Ya puedes enviar correos con los reportes!")

📄 Archivo .env existe
📋 Contenido actual del .env:
# Google Cloud Platform - Credenciales
GOOGLE_SERVICE_ACCOUNT_FILE=sistemaasistencia-470019-3f3d35327bca.json
GOOGLE_SHEET_ID=14AtL1_MHWaN1JR_dbbddif9w0ujlx4wFrEuelmv6gfs
SHEET_NAMES=REGISTRO_DIARIO,REGISTRO_CALENDARIO
DEFAULT_SHEET_NAME=REGISTRO_DIARIO

# Configuracion Gmail para envio de reportes
GMAIL_USER=proyectosperi@gmail.com
GMAIL_APP_PASSWORD=nhdz wqvv lxwc mgto

⚠️  Las credenciales de Gmail ya existen en .env
   Revisa manualmente el archivo para actualizarlas si es necesario

🔐 Configuración de Gmail:
   GMAIL_USER: proyectosperi@gmail.com
   GMAIL_APP_PASSWORD: nhdz wqvv lxwc mgto

🚀 ¡Ya puedes enviar correos con los reportes!
